<a href="https://colab.research.google.com/github/DaeunKim9/Smart-factory/blob/main/%E1%84%80%E1%85%A2%E1%84%8B%E1%85%B5%E1%86%AB%E1%84%91%E1%85%B3%E1%84%85%E1%85%A9%E1%84%8C%E1%85%A6%E1%86%A8%E1%84%90%E1%85%B3/%EA%B0%9C%EC%9D%B8%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import packages
from PIL import Image
import numpy as np
import os
import glob
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [3]:
# 데이터 경로
glob.glob('/content/drive/MyDrive/SW 전문인재양성 2기/개인프로젝트/CNN/mvtec_anomaly_detection/bottle/*')

['/content/drive/MyDrive/SW 전문인재양성 2기/개인프로젝트/CNN/mvtec_anomaly_detection/bottle/readme.txt',
 '/content/drive/MyDrive/SW 전문인재양성 2기/개인프로젝트/CNN/mvtec_anomaly_detection/bottle/license.txt',
 '/content/drive/MyDrive/SW 전문인재양성 2기/개인프로젝트/CNN/mvtec_anomaly_detection/bottle/ground_truth',
 '/content/drive/MyDrive/SW 전문인재양성 2기/개인프로젝트/CNN/mvtec_anomaly_detection/bottle/train',
 '/content/drive/MyDrive/SW 전문인재양성 2기/개인프로젝트/CNN/mvtec_anomaly_detection/bottle/test']

In [4]:
# 기본 경로
base_dir = '/content/drive/MyDrive/SW 전문인재양성 2기/개인프로젝트/CNN/mvtec_anomaly_detection/bottle'

In [5]:
train_dir_train_good_1 = os.path.join(base_dir, 'train', 'good')
train_dir_train_good_2 = os.path.join(base_dir, 'test','good')

test_dir_train_bad_1 = os.path.join(base_dir, 'test','broken_large')
test_dir_train_bad_2 = os.path.join(base_dir, 'test','broken_small')
test_dir_train_bad_3 = os.path.join(base_dir, 'test','contamination')

In [6]:
bottle_good = os.listdir(os.path.join(train_dir_train_good_1)) + (os.listdir(train_dir_train_good_2))
bottle_bad = os.listdir(os.path.join(test_dir_train_bad_1)) + os.listdir(os.path.join(test_dir_train_bad_2)) + (os.listdir(test_dir_train_bad_3))

print(bottle_good[:5])
print(bottle_bad [:5])

['022.png', '012.png', '009.png', '037.png', '033.png']
['003.png', '008.png', '016.png', '009.png', '018.png']


In [7]:
print('Total training good bottles images :', len(os.listdir(os.path.join(train_dir_train_good_1)) + (os.listdir(train_dir_train_good_2))))
print('Total training dog images :', len(os.listdir(os.path.join(test_dir_train_bad_1)) + os.listdir(os.path.join(test_dir_train_bad_2)) + (os.listdir(test_dir_train_bad_3))))


Total training good bottles images : 229
Total training dog images : 63


In [8]:
nrows, ncols = 4, 4
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*3, nrows*3)

pic_index+=8

next_cat_pix = [(os.listdir(os.path.join(train_dir_train_good_1)) + (os.listdir(train_dir_train_good_2)), fname)
                for fname in bottle_good[ pic_index-8:pic_index]]

next_dog_pix = [(os.listdir(os.path.join(test_dir_train_bad_1)) + os.listdir(os.path.join(test_dir_train_bad_2)) + (os.listdir(test_dir_train_bad_3))
, fname)
                for fname in bottle_bad[ pic_index-8:pic_index]]

# for i, img_path in enumerate(next_cat_pix+next_dog_pix):
#   sp = plt.subplot(nrows, ncols, i + 1)
#   sp.axis('Off')

#   img = mpimg.imread(img_path)
#   plt.imshow(img)

# plt.show()

<Figure size 1200x1200 with 0 Axes>

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 2)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 16)        304       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 10, 10, 64)      

In [14]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics = ['accuracy'])

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(base_dir,
                                                  batch_size=20,
                                                  class_mode='binary',
                                                  target_size=(150, 150))

Found 355 images belonging to 3 classes.


In [17]:
history = model.fit(train_generator,
                    steps_per_epoch=100,
                    epochs=100,
                    verbose=2)

Epoch 1/100


InvalidArgumentError: ignored